In [1]:
from mand.core import Entity, node, getNode
from mand.core import ObjectDb, Timestamp, Context, SimplificationContext, _tr
from mand.core import ProfileMonitor, SummaryMonitor
from mand.core import find, addFootnote
from mand.core import DependencyManager, setDependencyManager
from mand.demos.trading import makeWorld, bookSomeTrades, PnLExplainReport
from mand.lib.dbsetup import setUpDb

db = ObjectDb()
setUpDb(db)

In [2]:
small = False

with db:
    pWorld = makeWorld(small=small)
    
pAll, bExt, bExt2 = pWorld.children()
p1 = pAll.children()[0]
p2 = pAll.children()[1]
p4 = pAll.children()[3]

b2 = p2.children()[0]
b4 = p4.children()[0]

makeWorld, TopOfTheHouse is: <Entity:/Global/TradingPortfolio/TopOfTheHouse>
    # books: 100
    # children: 10


In [3]:
ts0, ts1, ts2, ts3, ts4, ts5, eod, ts6 = bookSomeTrades(pWorld, small=small)

p1.setChildren(p1.children() + [b2])
ts7 = Timestamp()

# Sanity check of a computation we might want to optimize [Test]

The PnL report is a good test, but at 152K getValues, it's a bit too big to immediately use:

In [4]:
db3 = db.copy()
p = db3._get(pAll.meta.path())

r = PnLExplainReport(valuable=p, ts1=eod, ts2=ts7)

with ProfileMonitor():
    with SummaryMonitor():
        r.run()

1. Inadequate cash discounting model used
1. Book appears multiple times
  * /Global/TradingBook/Eq-Inst0

# PnL explain for TopOfTheHouse: 6256.00

**Caveat: this report encountered problems. See footnotes at bottom.**

|Activity|PnL|
|-|-|
|Starting balance breaks|0.00
|prior day amends: MarketData|0.00
|prior day amends: Portfolio|0.00
|prior day amends: Trading|-4.00
|activity: MarketData|5240.00
|activity: Portfolio|1020.00
|activity: Trading|0.00
|Ending balance breaks|0.00

1. Inadequate cash discounting model used
1. Book appears multiple times
  * /Global/TradingBook/Eq-Inst0

### Compute activity summary (15.50 seconds of wall clock time)

|key|value|
|-|-|
|Context|11
|Db.Get|2176
|GetValue|152158
|GetValue/Calc|2891


### Profile by nodes.
* times are in microseconds
* cumT is total time spent in function
* calcT is time spent in function, but not in a child node

|fn|n|cumT|calcT|cumT/call|sys|
|-|-|-|-|-|-|
|PnLExplainReport:data|1|15,492,642|15|15,492,642|GetValue
|PnLExplainReport:data|1|15,492,627|562|15,492,627|GetValue/Calc
|TradingContainer:NPV|11|15,433,715|127|1,403,065|GetValue
|TradingContainer:NPV|11|15,433,588|77|1,403,053|GetValue/Calc
|Portfolio:items|121|15,290,261|1,220|126,365|GetValue
|Portfolio:items|121|15,290,160|873|126,364|GetValue/Calc
|Workbook:items|1,104|14,413,515|11,902|13,055|GetValue
|Workbook:items|1,100|14,401,612|8,310|13,092|GetValue/Calc
|PnLExplainReport:cutoffs|1|8,688,384|46|8,688,384|GetValue
|PnLExplainReport:cutoffs|1|8,688,338|218|8,688,338|GetValue/Calc
|Root:Clocks|548|8,688,009|1,401,969|15,854|Context
|TradeOpenEvent:ticket|22,286|3,433,654|183,320|154|GetValue
|TradeOpenEvent|1,014|3,321,260|3,321,260|3,275|Db.Get
|TradingTicket|1,014|3,250,334|3,250,334|3,205|Db.Get
|Root:End|278|946,514|870,219|3,404|Context
|Portfolio:children|242|840,169|1,689|3,471|GetValue
|Portfolio:children|121|838,479|883|6,929|GetValue/Calc
|Root:Activity MarketData|255|802,445|731,050|3,146|Context
|Root:Amend Portfolio|255|782,711|709,421|3,069|Context
|Root:Start|270|771,393|699,526|2,857|Context
|Root:Amend MarketData|255|760,772|689,758|2,983|Context
|Root:Activity Trading|263|697,257|624,687|2,651|Context
|Root:Amend Trading|255|685,992|614,175|2,690|Context
|Root:Activity Portfolio|263|677,298|607,886|2,575|Context
|Root:Start breaks|255|674,220|602,706|2,644|Context
|PortfolioUpdateEvent:children|121|358,972|2,690|2,966|GetValue
|TradingBook|102|333,460|333,460|3,269|Db.Get
|Equity:NPV|15|142,142|160|9,476|GetValue
|Equity:NPV|15|141,981|105|9,465|GetValue/Calc
|MarketInterface:spot|15|133,897|155|8,926|GetValue
|MarketInterface:spot|15|133,741|104|8,916|GetValue/Calc
|ExternalRefData:state|15|125,730|149|8,382|GetValue
|ExternalRefData:state|15|125,581|101|8,372|GetValue/Calc
|RefData:state|15|125,104|143|8,340|GetValue
|RefData:state|15|124,961|107|8,330|GetValue/Calc
|_WorkItemEvent:book2|22,286|107,925|102,296|4|GetValue
|_WorkItemEvent:book1|22,286|105,357|105,357|4|GetValue
|TradeOpenEvent:quantity|22,286|100,941|100,941|4|GetValue
|Clock:cutoffs|2,480|92,107|9,874|37|GetValue
|Clock:cutoffs|20|82,317|144|4,115|GetValue/Calc
|Clock:parent|20|81,303|208|4,065|GetValue
|Clock:parent|20|81,095|144|4,054|GetValue/Calc
|_WorkItemEvent:item|11,143|74,871|69,024|6|GetValue
|TradeOpenEvent:premium|11,143|65,872|63,052|5|GetValue
|TradeOpenEvent:action|11,143|63,059|63,059|5|GetValue
|Event:amends|11,340|57,940|57,940|5|GetValue
|TradeOpenEvent:unitPrice|11,143|55,802|55,802|5|GetValue
|TradingBook:clock|2,200|52,425|20,691|23|GetValue
|PortfolioUpdateEvent|12|45,277|45,277|3,773|Db.Get
|RefDataUpdateEvent|9|32,420|32,420|3,602|Db.Get
|TradingBook:clock|1,100|31,734|10,573|28|GetValue/Calc
|TradingPortfolio|10|28,450|28,450|2,845|Db.Get
|Clock|5|16,015|16,015|3,203|Db.Get
|Portfolio:clock|242|8,482|2,278|35|GetValue
|Portfolio:books|231|7,580|1,708|32|GetValue
|Equity:refdata|15|7,376|148|491|GetValue
|MarketInterface:source|15|7,249|134|483|GetValue
|Equity:refdata|15|7,228|105|481|GetValue/Calc
|MarketInterface:source|15|7,114|98|474|GetValue/Calc
|ClockEvent|2|6,777|6,777|3,388|Db.Get
|MarketInterface|2|6,441|6,441|3,220|Db.Get
|ClockEvent:parent|8|6,391|76|798|GetValue
|MarketDataSource|2|6,268|6,268|3,134|Db.Get
|Portfolio:books|121|6,258|887|51|GetValue/Calc
|Portfolio:clock|121|6,204|1,142|51|GetValue/Calc
|Equity|2|5,846|5,846|2,923|Db.Get
|RootClock|1|5,060|5,060|5,060|Db.Get
|MarketDataSource:clock|30|4,252|280|141|GetValue
|MarketDataSource:clock|15|3,971|149|264|GetValue/Calc
|ForwardCashflow|1|2,820|2,820|2,820|Db.Get
|Entity:clock|40|837|361|20|GetValue
|Entity:clock|20|475|188|23|GetValue/Calc
|ForwardCashflow:NPV|11|324|142|29|GetValue
|RootClock:cutoffs|53|308|222|5|GetValue
|MarketInterface:sourceName|15|308|135|20|GetValue
|RefDataUpdateEvent:data|58|277|277|4|GetValue
|ForwardCashflow:NPV|11|182|77|16|GetValue/Calc
|MarketInterface:sourceName|15|172|104|11|GetValue/Calc
|Equity:assetName|15|92|92|6|GetValue
|RootClock:cutoffs|1|85|43|85|GetValue/Calc
|RootClock:cosmicAll|1|25|12|25|GetValue
|CosmicAll:dbState|1|16|10|16|GetValue
|RootClock:cosmicAll|1|13|13|13|GetValue/Calc
|PnLExplainReport:valuable|2|9|9|4|GetValue
|PnLExplainReport:ts2|2|9|9|4|GetValue
|PnLExplainReport:ts1|2|7|7|3|GetValue
|CosmicAll:dbState|1|5|5|5|GetValue/Calc

# Caching/Reusing results

Until now, we have just checked to see if the current context contains a value for a node we are asking for, and if so, reuse that.

A better approach to caching bound *fn* on *object* in *context0* is:
    
1. Has fn been tweaked in context0? If so, return that
2. Is fn sufficiently trivial that we can avoid managing it as a node?
  * If trivial, just treat it as a pure python function and call it
  * Meta-data (inputs, outputs, etc) will be given to its caller and callees as appropriate
3. Ask object for context1
  * *context1* is a simplified version of *context0*
  * The default case is just to return *context0*
  * IRL, this could actually be a list of contexts or a pattern to match contexts against
4. Is fn cached in *context1*? If so, use that
5. Compute *fn* in *context1*
6. Construct *context2* from the inputs of the computed value
7. If *context2* is a subset of *context1*, cache *fn* in *context1*
8. Something odd happened
  * Footnote the problem as part of computation notes on the node's metadata
  * Maybe cache the node anyway in *context2*
    * Perhaps *object* will return *context2* as a simplification for future computations?
9. Return the value

Notes:
* Parallel compute of nodes not considered yet
* Context simplification (step 3) and input simplification (step 9) are probably intimately related
* The split between calculation and caching is nice:
  * BAs can write business logic
  * Computer scientists can add caching logic where needed
* We can use a profiler type object to gather runtime compute cost infomation
  * The resulting trace can be used as input to drive step 2 and step 3
    

# Graph Simplification

* Very simple input management simplifier: ignore inputs if they are leaves and not tweakable
* Simple context simplification: just switch to a simple context for certain known methods

In [5]:
class DM1(DependencyManager):
    
    def __init__(self):
        self.contexts = {}
        super(DM1, self).__init__()
        
    def addDep(self, input, output):
        if not input.key.tweakable:
            if not input.inputs:
                return
        output.inputs.add(input)
        input.outputs.add(output)

    def calculated(self, node):
        if not node.isSimplified:
            return True
        ok = True
        for input in node.floatingTweakPoints():
            addFootnote(text='context simplification failure', info='%s in %s on %s in %s' % (str(node.key), node.ctx.name, str(input.key), input.ctx.name))
            ok = False
        return ok

    def simplify(self, key):
        if key.fullName() == 'RefData:state':
            obj = key.object()
            return (obj.clock().cutoffs,)
        if key.fullName() == 'Portfolio:items':
            obj = key.object()
            obj2 = obj.getObj(_tr.Clock, 'Trading')
            return (obj.clock().cutoffs, obj2.cutoffs)
        if key.fullName() == 'Workbook:items':
            obj = key.object()
            return (obj.clock().cutoffs,)
        
    def getNode(self, ctx, key):
        n = ctx._get(key)
        if n:
            return n
        s = self.simplify(key)
        if s is not None: 
            nodes = [ getNode(bm) for bm in s ]
            values = [ bm() for bm in s ]
            k = [ (n.object(), n.key.fullName(), v) for n, v in zip(nodes, values) ]
            cKey = tuple(k)
            
            if cKey not in self.contexts:
                tweaks = dict( [ (bm, v) for bm, v in zip(s, values)])
                name = 'simp-%s' % len(self.contexts)
                ctx1 = SimplificationContext(tweaks, name)
                self.contexts[cKey] = ctx1
            ctx1 = self.contexts[cKey]
            ret = ctx1.get(key)
            ret.isSimplified = True
        else:
            ret = super(DM1, self).getNode(ctx, key)
            ret.isSimplified = False
        return ret
    
dm = DM1()
setDependencyManager(dm)


In [6]:
db4 = db.copy()
valuable = db4.get(pAll.meta.path())

r = PnLExplainReport(valuable=valuable, ts1=eod, ts2=ts7)
with ProfileMonitor():
    with SummaryMonitor():
        r.run()

1. Inadequate cash discounting model used
1. Book appears multiple times
  * /Global/TradingBook/Eq-Inst0

# PnL explain for TopOfTheHouse: 6256.00

**Caveat: this report encountered problems. See footnotes at bottom.**

|Activity|PnL|
|-|-|
|Starting balance breaks|0.00
|prior day amends: MarketData|0.00
|prior day amends: Portfolio|0.00
|prior day amends: Trading|-4.00
|activity: MarketData|5240.00
|activity: Portfolio|1020.00
|activity: Trading|0.00
|Ending balance breaks|0.00

1. Inadequate cash discounting model used
1. Book appears multiple times
  * /Global/TradingBook/Eq-Inst0

### Compute activity summary (10.99 seconds of wall clock time)

|key|value|
|-|-|
|Context|22
|Db.Get|2176
|GetValue|43372
|GetValue/Calc|1531


### Profile by nodes.
* times are in microseconds
* cumT is total time spent in function
* calcT is time spent in function, but not in a child node

|fn|n|cumT|calcT|cumT/call|sys|
|-|-|-|-|-|-|
|PnLExplainReport:data|1|10,982,651|15|10,982,651|GetValue
|PnLExplainReport:data|1|10,982,636|561|10,982,636|GetValue/Calc
|TradingContainer:NPV|11|10,976,183|128|997,834|GetValue
|TradingContainer:NPV|11|10,976,054|83|997,823|GetValue/Calc
|Portfolio:items|61|10,799,427|659|177,039|GetValue
|Portfolio:items|55|10,799,340|425|196,351|GetValue/Calc
|Workbook:items|502|10,104,226|4,602|20,127|GetValue
|Workbook:items|300|10,099,623|2,455|33,665|GetValue/Calc
|PnLExplainReport:cutoffs|1|9,772,989|15|9,772,989|GetValue
|PnLExplainReport:cutoffs|1|9,772,974|342|9,772,974|GetValue/Calc
|Root:Clocks|59|9,772,521|40,978|165,635|Context
|simp-0|144|8,816,101|53,903|61,222|Context
|simp-1|200|8,346,103|714,519|41,730|Context
|TradeOpenEvent|1,014|3,788,760|3,788,760|3,736|Db.Get
|TradeOpenEvent:ticket|6,078|3,776,288|44,909|621|GetValue
|TradingTicket|1,014|3,731,379|3,731,379|3,679|Db.Get
|Root:Amend Trading|11|1,045,138|501|95,012|Context
|simp-8|144|1,044,401|51,777|7,252|Context
|simp-9|200|986,709|910,071|4,933|Context
|simp-3|144|825,889|55,148|5,735|Context
|simp-4|200|764,355|681,051|3,821|Context
|Portfolio:children|110|626,362|958|5,694|GetValue
|Portfolio:children|55|625,404|509|11,370|GetValue/Calc
|PortfolioUpdateEvent:children|55|364,999|1,932|6,636|GetValue
|TradingBook|102|334,408|334,408|3,278|Db.Get
|Equity:NPV|15|115,542|172|7,702|GetValue
|Equity:NPV|15|115,370|116|7,691|GetValue/Calc
|MarketInterface:spot|15|107,023|159|7,134|GetValue
|MarketInterface:spot|15|106,863|130|7,124|GetValue/Calc
|ExternalRefData:state|15|98,786|158|6,585|GetValue
|ExternalRefData:state|15|98,628|113|6,575|GetValue/Calc
|Clock:cutoffs|1,365|95,939|5,931|70|GetValue
|Clock:cutoffs|20|90,094|158|4,504|GetValue/Calc
|Clock:parent|20|88,967|205|4,448|GetValue
|Clock:parent|20|88,761|161|4,438|GetValue/Calc
|Root:Activity Portfolio|18|58,790|814|3,266|Context
|simp-10|144|57,585|53,062|399|Context
|Root:Amend Portfolio|11|56,021|476|5,092|Context
|simp-7|144|55,322|51,192|384|Context
|RefData:state|15|54,904|102|3,660|GetValue
|RefData:state|4|54,802|32|13,700|GetValue/Calc
|PortfolioUpdateEvent|12|46,118|46,118|3,843|Db.Get
|RefDataUpdateEvent|9|38,830|38,830|4,314|Db.Get
|_WorkItemEvent:book2|6,078|37,783|31,541|6|GetValue
|TradingPortfolio|10|34,900|34,900|3,490|Db.Get
|_WorkItemEvent:book1|6,078|33,499|33,499|5|GetValue
|_WorkItemEvent:item|3,039|32,705|25,965|10|GetValue
|TradeOpenEvent:quantity|6,078|31,691|31,691|5|GetValue
|TradingBook:clock|1,102|31,553|12,179|28|GetValue
|simp-5|4|27,891|7,244|6,972|Context
|simp-2|2|23,055|4,621|11,527|Context
|TradeOpenEvent:premium|3,039|22,968|20,001|7|GetValue
|TradeOpenEvent:action|3,039|20,788|20,788|6|GetValue
|Root:End|33|20,390|19,604|617|Context
|TradingBook:clock|800|19,373|7,268|24|GetValue/Calc
|Root:Start|26|18,905|18,288|727|Context
|Event:amends|3,122|18,006|18,006|5|GetValue
|TradeOpenEvent:unitPrice|3,039|17,590|17,590|5|GetValue
|Clock|5|16,890|16,890|3,378|Db.Get
|Equity:refdata|15|7,476|167|498|GetValue
|ClockEvent|2|7,372|7,372|3,686|Db.Get
|Equity:refdata|15|7,308|258|487|GetValue/Calc
|MarketInterface:source|15|7,253|154|483|GetValue
|ClockEvent:parent|8|7,143|82|892|GetValue
|MarketInterface:source|15|7,098|109|473|GetValue/Calc
|Equity|2|6,739|6,739|3,369|Db.Get
|MarketInterface|2|6,448|6,448|3,224|Db.Get
|MarketDataSource|2|6,170|6,170|3,085|Db.Get
|Portfolio:clock|171|5,953|1,425|34|GetValue
|Portfolio:clock|66|4,528|517|68|GetValue/Calc
|Root:Amend MarketData|11|4,462|423|405|Context
|MarketDataSource:clock|23|4,212|233|183|GetValue
|MarketDataSource:clock|19|3,978|163|209|GetValue/Calc
|simp-6|2|3,822|3,748|1,911|Context
|Portfolio:books|105|3,721|855|35|GetValue
|RootClock|1|3,107|3,107|3,107|Db.Get
|Portfolio:books|55|3,060|447|55|GetValue/Calc
|ForwardCashflow|1|2,966|2,966|2,966|Db.Get
|Root:Activity Trading|18|995|649|55|Context
|Entity:clock|40|947|412|23|GetValue
|Root:Start breaks|11|632|424|57|Context
|Root:Activity MarketData|11|628|416|57|Context
|Entity:clock|20|535|216|26|GetValue/Calc
|ForwardCashflow:NPV|11|344|149|31|GetValue
|MarketInterface:sourceName|15|338|156|22|GetValue
|RootClock:cutoffs|53|315|226|5|GetValue
|ForwardCashflow:NPV|11|195|88|17|GetValue/Calc
|MarketInterface:sourceName|15|181|109|12|GetValue/Calc
|Equity:assetName|15|93|93|6|GetValue
|RootClock:cutoffs|1|89|49|89|GetValue/Calc
|RefDataUpdateEvent:data|15|77|77|5|GetValue
|RootClock:cosmicAll|1|23|14|23|GetValue
|CosmicAll:dbState|1|16|10|16|GetValue
|PnLExplainReport:valuable|2|8|8|4|GetValue
|RootClock:cosmicAll|1|8|8|8|GetValue/Calc
|PnLExplainReport:ts1|2|8|8|4|GetValue
|PnLExplainReport:ts2|2|6|6|3|GetValue
|CosmicAll:dbState|1|6|6|6|GetValue/Calc

In [7]:
for k, ctx in sorted(dm.contexts.items(), key=lambda x: x[1].name):
    print()
    print(ctx.name)
    for t in k:
        print('    ', t)
    for t in ctx.tweaks:
        print('        ', t)


simp-0
     (<Entity:/Global/Clock/Portfolio>, 'Clock:cutoffs', <TS:t=2017-05-31T11:16:37.458351,v=2017-05-31T11:16:37.458350>)
     (<Entity:/Global/Clock/Trading>, 'Clock:cutoffs', <TS:t=2017-05-31T11:16:37.458351,v=2017-05-31T11:16:37.458350>)
         <Node: /Global/Clock/Portfolio.(Clock:cutoffs)() in simp-0 @4571832728>
         <Node: /Global/Clock/Trading.(Clock:cutoffs)() in simp-0 @4571832784>

simp-1
     (<Entity:/Global/Clock/Trading>, 'Clock:cutoffs', <TS:t=2017-05-31T11:16:37.458351,v=2017-05-31T11:16:37.458350>)
         <Node: /Global/Clock/Trading.(Clock:cutoffs)() in simp-1 @4399401672>

simp-10
     (<Entity:/Global/Clock/Portfolio>, 'Clock:cutoffs', <TS:t=2017-05-31T11:16:37.512334,v=2017-05-31T11:16:37.512333>)
     (<Entity:/Global/Clock/Trading>, 'Clock:cutoffs', <TS:t=2017-05-31T11:16:37.512334,v=2017-05-31T11:16:37.458350>)
         <Node: /Global/Clock/Trading.(Clock:cutoffs)() in simp-10 @4611482120>
         <Node: /Global/Clock/Portfolio.(Clock:cutoffs)()